# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/11_series_temporelles/11_exercices.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '11_exercices.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 12 - Séries Temporelles : Exercices

Ce notebook contient 3 exercices pratiques avec solutions complètes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Séries temporelles
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

## Exercice 1 : Prédiction de Ventes Mensuelles avec ARIMA

**Contexte :** Vous travaillez pour un retailer qui souhaite prédire ses ventes mensuelles.

**Objectifs :**
1. Générer des ventes mensuelles synthétiques avec tendance et saisonnalité
2. Analyser la série (décomposition, stationnarité)
3. Ajuster un modèle ARIMA
4. Faire des prédictions sur 12 mois
5. Évaluer avec RMSE et MAPE

---

### Données

In [ ]:
# TODO: Générer ventes mensuelles (5 ans = 60 mois)
# - Tendance croissante: +2% par mois
# - Saisonnalité annuelle (pic en décembre)
# - Bruit gaussien

def generate_monthly_sales(n_months=60, base_sales=10000, trend_rate=0.02, noise_std=500, seed=42):
    # TODO: Implémenter
    pass

# Générer
# sales_df = generate_monthly_sales(n_months=60)

# TODO: Visualiser la série

### Analyse Exploratoire

In [ ]:
# TODO: Décomposition de la série (trend, seasonality, residual)
# Utiliser seasonal_decompose avec period=12

In [ ]:
# TODO: Test de stationnarité (ADF test)
# Si non-stationnaire, différencier la série

### Modèle ARIMA

In [ ]:
# TODO: Split train/test (80/20)
# TODO: Sélectionner paramètres ARIMA (p, d, q)
# TODO: Entraîner ARIMA
# TODO: Prédire sur test set
# TODO: Calculer RMSE et MAPE

---

## SOLUTION Exercice 1

---

In [ ]:
# SOLUTION: Génération des données
def generate_monthly_sales(n_months=60, base_sales=10000, trend_rate=0.02, noise_std=500, seed=42):
    np.random.seed(seed)
    
    dates = pd.date_range(start='2019-01-01', periods=n_months, freq='MS')
    t = np.arange(n_months)
    
    # Tendance
    trend = base_sales * (1 + trend_rate) ** t
    
    # Saisonnalité (pic en décembre = mois 11)
    seasonality = 2000 * np.sin(2 * np.pi * t / 12 - np.pi/2)  # Max au mois 12
    
    # Bruit
    noise = np.random.normal(0, noise_std, n_months)
    
    # Ventes
    sales = trend + seasonality + noise
    
    df = pd.DataFrame({
        'date': dates,
        'sales': sales
    })
    df.set_index('date', inplace=True)
    
    return df

# Générer
sales_df = generate_monthly_sales(n_months=60)

print(f"Ventes générées: {len(sales_df)} mois")
print(sales_df.head())

# Visualisation
plt.figure(figsize=(14, 5))
plt.plot(sales_df.index, sales_df['sales'], marker='o', color='blue')
plt.title('Ventes Mensuelles - 5 ans', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Ventes ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# SOLUTION: Décomposition
decomposition = seasonal_decompose(sales_df['sales'], model='additive', period=12)

fig, axes = plt.subplots(4, 1, figsize=(14, 10))
decomposition.observed.plot(ax=axes[0], title='Ventes Observées', color='blue')
decomposition.trend.plot(ax=axes[1], title='Tendance', color='green')
decomposition.seasonal.plot(ax=axes[2], title='Saisonnalité', color='orange')
decomposition.resid.plot(ax=axes[3], title='Résidus', color='red')
plt.tight_layout()
plt.show()

In [ ]:
# SOLUTION: Test ADF
result = adfuller(sales_df['sales'])
print(f"ADF Statistic: {result[0]:.6f}")
print(f"p-value: {result[1]:.6f}")

if result[1] < 0.05:
    print("✅ Série stationnaire")
else:
    print("❌ Série non-stationnaire -> différenciation nécessaire")

# Différenciation
sales_diff = sales_df['sales'].diff().dropna()
result_diff = adfuller(sales_diff)
print(f"\nAprès différenciation:")
print(f"ADF Statistic: {result_diff[0]:.6f}")
print(f"p-value: {result_diff[1]:.6f}")

In [ ]:
# SOLUTION: ACF et PACF
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(sales_diff, lags=20, ax=axes[0])
plot_pacf(sales_diff, lags=20, ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
# SOLUTION: ARIMA
train_size = int(len(sales_df) * 0.8)
train = sales_df['sales'][:train_size]
test = sales_df['sales'][train_size:]

print(f"Train: {len(train)} mois")
print(f"Test: {len(test)} mois")

# ARIMA(1,1,1) basé sur ACF/PACF
model = ARIMA(train, order=(1, 1, 1))
fitted = model.fit()

print(f"\nAIC: {fitted.aic:.2f}")
print(f"BIC: {fitted.bic:.2f}")

# Prédictions
forecast = fitted.forecast(steps=len(test))

# Métriques
rmse = np.sqrt(mean_squared_error(test, forecast))
mape = np.mean(np.abs((test - forecast) / test)) * 100

print(f"\n=== Métriques ===")
print(f"RMSE: {rmse:.2f}$")
print(f"MAPE: {mape:.2f}%")

# Visualisation
plt.figure(figsize=(14, 6))
plt.plot(train.index, train, label='Train', color='blue')
plt.plot(test.index, test, label='Test (Réel)', color='green', marker='o')
plt.plot(test.index, forecast, label='Prédictions ARIMA', color='red', linestyle='--', marker='x')
plt.axvline(train.index[-1], color='black', linestyle=':', label='Train/Test Split')
plt.title('Prédiction Ventes Mensuelles - ARIMA(1,1,1)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Ventes ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## Exercice 2 : Forecasting de Consommation Électrique avec LSTM

**Contexte :** Prédire la consommation électrique horaire en utilisant des features multivariées.

**Objectifs :**
1. Créer une série multivariée (consommation, température, heure du jour, jour de la semaine)
2. Feature engineering (lags, rolling stats)
3. Créer des sliding windows
4. Entraîner un LSTM
5. Évaluer les prédictions

---

### Données

In [ ]:
# TODO: Générer données horaires sur 30 jours
# Features:
# - temperature: 15-30°C avec variation journalière
# - hour: 0-23
# - weekday: 0-6
# - consumption: corrélée avec température et heure (pic en journée)

### Feature Engineering et Preprocessing

In [ ]:
# TODO: Créer features supplémentaires
# - Lags de consommation (lag 1, 24, 168)
# - Rolling mean (fenêtre 24h)
# - Features cycliques pour hour (sin/cos)

### LSTM Multivarié

In [ ]:
# TODO: Créer sliding windows avec features multivariées
# TODO: Normaliser les données
# TODO: Définir architecture LSTM
# TODO: Entraîner
# TODO: Évaluer

---

## SOLUTION Exercice 2

---

In [ ]:
# SOLUTION: Génération données
def generate_energy_consumption(n_hours=30*24, seed=42):
    np.random.seed(seed)
    
    dates = pd.date_range(start='2024-01-01', periods=n_hours, freq='H')
    
    # Features temporelles
    hour = dates.hour
    weekday = dates.weekday
    
    # Température (variation journalière + bruit)
    t = np.arange(n_hours)
    temp = 20 + 5 * np.sin(2 * np.pi * t / 24 - np.pi/2) + np.random.normal(0, 1, n_hours)
    
    # Consommation (dépend de heure et température)
    # Base + pic journée (8h-20h) + effet température + weekend
    consumption_base = 5000
    hour_effect = 1500 * np.sin(2 * np.pi * (hour - 6) / 24)
    hour_effect[hour_effect < 0] = 0
    temp_effect = 50 * (temp - 20)
    weekend_effect = -500 * ((weekday == 5) | (weekday == 6))
    noise = np.random.normal(0, 200, n_hours)
    
    consumption = consumption_base + hour_effect + temp_effect + weekend_effect + noise
    
    df = pd.DataFrame({
        'date': dates,
        'temperature': temp,
        'hour': hour,
        'weekday': weekday,
        'consumption': consumption
    })
    df.set_index('date', inplace=True)
    
    return df

energy_df = generate_energy_consumption(n_hours=30*24)

print(f"Données générées: {len(energy_df)} heures")
print(energy_df.head(10))

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(energy_df.index, energy_df['consumption'], color='blue', linewidth=0.8)
axes[0].set_title('Consommation Électrique Horaire', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Consommation (kWh)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(energy_df.index, energy_df['temperature'], color='red', linewidth=0.8)
axes[1].set_title('Température', fontsize=12)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Température (°C)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# SOLUTION: Feature Engineering
df_features = energy_df.copy()

# Lags
df_features['consumption_lag1'] = df_features['consumption'].shift(1)
df_features['consumption_lag24'] = df_features['consumption'].shift(24)

# Rolling mean (24h)
df_features['consumption_rolling_mean_24'] = df_features['consumption'].rolling(window=24).mean()

# Features cycliques pour heure
df_features['hour_sin'] = np.sin(2 * np.pi * df_features['hour'] / 24)
df_features['hour_cos'] = np.cos(2 * np.pi * df_features['hour'] / 24)

# Supprimer NaN
df_features = df_features.dropna()

print(f"Features créées: {df_features.shape[1]} colonnes")
print(df_features.columns.tolist())
print(f"\nDonnées après feature engineering: {len(df_features)} heures")

In [ ]:
# SOLUTION: Préparation pour LSTM
# Features: toutes sauf consumption (target)
feature_cols = ['temperature', 'hour', 'weekday', 'consumption_lag1', 
                'consumption_lag24', 'consumption_rolling_mean_24', 
                'hour_sin', 'hour_cos']

X = df_features[feature_cols].values
y = df_features['consumption'].values

# Train/Val/Test split
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train = X[:train_size]
y_train = y[:train_size]
X_val = X[train_size:train_size+val_size]
y_val = y[train_size:train_size+val_size]
X_test = X[train_size+val_size:]
y_test = y[train_size+val_size:]

# Normalisation
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

# Sliding windows
def create_sequences_multivariate(X, y, window_size):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size):
        X_seq.append(X[i:i+window_size])
        y_seq.append(y[i+window_size])
    return np.array(X_seq), np.array(y_seq)

WINDOW_SIZE = 24  # 24h

X_train_seq, y_train_seq = create_sequences_multivariate(X_train_scaled, y_train_scaled, WINDOW_SIZE)
X_val_seq, y_val_seq = create_sequences_multivariate(X_val_scaled, y_val_scaled, WINDOW_SIZE)
X_test_seq, y_test_seq = create_sequences_multivariate(X_test_scaled, y_test_scaled, WINDOW_SIZE)

print(f"X_train_seq shape: {X_train_seq.shape}  # (samples, window, features)")
print(f"y_train_seq shape: {y_train_seq.shape}")

# Tensors
X_train_t = torch.FloatTensor(X_train_seq)
y_train_t = torch.FloatTensor(y_train_seq)
X_val_t = torch.FloatTensor(X_val_seq)
y_val_t = torch.FloatTensor(y_val_seq)
X_test_t = torch.FloatTensor(X_test_seq)
y_test_t = torch.FloatTensor(y_test_seq)

# DataLoaders
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# SOLUTION: Modèle LSTM Multivarié
class LSTMMultivariate(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2):
        super(LSTMMultivariate, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output.squeeze()

model = LSTMMultivariate(input_size=len(feature_cols), hidden_size=64, num_layers=2).to(device)

print(model)
print(f"\nParamètres: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

In [ ]:
# SOLUTION: Entraînement
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Train: {train_loss:.6f}, Val: {val_loss:.6f}")

print("\n✅ Entraînement terminé")

In [ ]:
# SOLUTION: Évaluation
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(batch_y.numpy())

predictions = np.array(predictions)
actuals = np.array(actuals)

# Inverse transform
predictions_orig = scaler_y.inverse_transform(predictions.reshape(-1, 1)).flatten()
actuals_orig = scaler_y.inverse_transform(actuals.reshape(-1, 1)).flatten()

# Métriques
mae = mean_absolute_error(actuals_orig, predictions_orig)
rmse = np.sqrt(mean_squared_error(actuals_orig, predictions_orig))
mape = np.mean(np.abs((actuals_orig - predictions_orig) / actuals_orig)) * 100

print("\n=== Métriques LSTM Multivarié ===")
print(f"MAE:  {mae:.2f} kWh")
print(f"RMSE: {rmse:.2f} kWh")
print(f"MAPE: {mape:.2f}%")

# Visualisation
plt.figure(figsize=(14, 6))
plt.plot(actuals_orig[:200], label='Réel', color='green', linewidth=1.5)
plt.plot(predictions_orig[:200], label='Prédictions LSTM', color='red', linestyle='--', alpha=0.8)
plt.title('Prédiction Consommation Électrique - LSTM Multivarié', fontsize=14, fontweight='bold')
plt.xlabel('Heures')
plt.ylabel('Consommation (kWh)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## Exercice 3 : Détection d'Anomalies dans une Série Temporelle

**Contexte :** Monitorer des transactions bancaires et détecter les fraudes.

**Objectifs :**
1. Générer série de transactions avec anomalies injectées
2. Entraîner LSTM pour prédire transactions normales
3. Détecter anomalies via erreur de prédiction
4. Évaluer avec précision/rappel si labels disponibles

---

---

## SOLUTION Exercice 3

---

In [ ]:
# SOLUTION: Génération avec anomalies
def generate_transactions_with_anomalies(n=1000, anomaly_rate=0.05, seed=42):
    np.random.seed(seed)
    
    # Transactions normales (pattern journalier + bruit)
    t = np.arange(n)
    base = 1000
    daily_pattern = 300 * np.sin(2 * np.pi * t / 24)
    noise = np.random.normal(0, 50, n)
    transactions = base + daily_pattern + noise
    
    # Injecter anomalies (valeurs extrêmes)
    n_anomalies = int(n * anomaly_rate)
    anomaly_indices = np.random.choice(n, size=n_anomalies, replace=False)
    
    labels = np.zeros(n)
    for idx in anomaly_indices:
        # Anomalie = transaction 3-5x supérieure à la normale
        transactions[idx] *= np.random.uniform(3, 5)
        labels[idx] = 1
    
    dates = pd.date_range(start='2024-01-01', periods=n, freq='H')
    
    df = pd.DataFrame({
        'date': dates,
        'amount': transactions,
        'is_anomaly': labels
    })
    df.set_index('date', inplace=True)
    
    return df

transactions_df = generate_transactions_with_anomalies(n=1000, anomaly_rate=0.05)

print(f"Transactions générées: {len(transactions_df)}")
print(f"Anomalies: {transactions_df['is_anomaly'].sum()} ({100*transactions_df['is_anomaly'].mean():.1f}%)")

# Visualisation
plt.figure(figsize=(14, 6))
normal = transactions_df[transactions_df['is_anomaly'] == 0]
anomalies = transactions_df[transactions_df['is_anomaly'] == 1]

plt.plot(normal.index, normal['amount'], color='blue', linewidth=0.8, label='Normal')
plt.scatter(anomalies.index, anomalies['amount'], color='red', s=100, marker='x', 
           label=f'Anomalies ({len(anomalies)})', zorder=5)
plt.title('Transactions Bancaires avec Anomalies', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Montant ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# SOLUTION: Préparation données
data = transactions_df['amount'].values
labels = transactions_df['is_anomaly'].values

# Normalisation
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.reshape(-1, 1)).flatten()

# Windows
WINDOW_SIZE = 24

def create_sequences(data, labels, window_size):
    X, y, y_labels = [], [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
        y_labels.append(labels[i+window_size])
    return np.array(X), np.array(y), np.array(y_labels)

X, y, y_labels = create_sequences(data_scaled, labels, WINDOW_SIZE)

# Split (70/30)
train_size = int(len(X) * 0.7)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]
y_test_labels = y_labels[train_size:]

# Tensors
X_train_t = torch.FloatTensor(X_train).unsqueeze(-1)
y_train_t = torch.FloatTensor(y_train)
X_test_t = torch.FloatTensor(X_test).unsqueeze(-1)
y_test_t = torch.FloatTensor(y_test)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=32, shuffle=False)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# SOLUTION: LSTM pour détection
class LSTMAnomaly(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1):
        super(LSTMAnomaly, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output.squeeze()

anomaly_model = LSTMAnomaly(input_size=1, hidden_size=32).to(device)

# Entraînement
criterion = nn.MSELoss()
optimizer = optim.Adam(anomaly_model.parameters(), lr=0.001)

num_epochs = 30
for epoch in range(num_epochs):
    anomaly_model.train()
    train_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        outputs = anomaly_model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader):.6f}")

print("\n✅ Entraînement terminé")

In [ ]:
# SOLUTION: Détection d'anomalies
anomaly_model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = anomaly_model(batch_X)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(batch_y.numpy())

predictions = np.array(predictions)
actuals = np.array(actuals)

# Erreur de prédiction
errors = np.abs(actuals - predictions)

# Seuil (moyenne + 3 std)
threshold = errors.mean() + 3 * errors.std()
detected_anomalies = errors > threshold

print(f"\n=== Détection d'Anomalies ===")
print(f"Seuil: {threshold:.6f}")
print(f"Anomalies détectées: {detected_anomalies.sum()} / {len(detected_anomalies)}")
print(f"Vraies anomalies (labels): {y_test_labels.sum()}")

# Évaluation si labels disponibles
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

precision = precision_score(y_test_labels, detected_anomalies)
recall = recall_score(y_test_labels, detected_anomalies)
f1 = f1_score(y_test_labels, detected_anomalies)

print(f"\nPrécision: {precision:.3f}")
print(f"Rappel:    {recall:.3f}")
print(f"F1-Score:  {f1:.3f}")

print(f"\nMatrice de Confusion:")
print(confusion_matrix(y_test_labels, detected_anomalies))

# Visualisation
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(actuals, label='Réel', color='blue', linewidth=1)
plt.plot(predictions, label='Prédictions', color='green', linestyle='--', alpha=0.7)
true_anomalies = np.where(y_test_labels == 1)[0]
detected = np.where(detected_anomalies)[0]
plt.scatter(true_anomalies, actuals[true_anomalies], color='red', s=100, marker='o', 
           label='Vraies anomalies', zorder=5)
plt.scatter(detected, actuals[detected], color='orange', s=50, marker='x', 
           label='Détectées', zorder=4)
plt.title('Détection d\'Anomalies - Transactions', fontsize=14, fontweight='bold')
plt.ylabel('Montant (normalisé)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(errors, color='purple', alpha=0.7)
plt.axhline(threshold, color='red', linestyle='--', linewidth=2, label=f'Seuil = {threshold:.4f}')
plt.scatter(detected, errors[detected], color='red', s=50, marker='x', zorder=5)
plt.title('Erreurs de Prédiction', fontsize=12)
plt.xlabel('Pas de temps')
plt.ylabel('Erreur absolue')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Conclusion des Exercices

### Exercice 1 - ARIMA
- Décomposition et test de stationnarité essentiels
- Différenciation pour stationnariser
- Sélection de (p,d,q) via ACF/PACF
- ARIMA efficace pour séries univariées avec patterns linéaires

### Exercice 2 - LSTM Multivarié
- Feature engineering crucial (lags, rolling stats, features cycliques)
- LSTM capte dépendances complexes entre features
- Normalisation et windowing appropriés
- Excellent pour forecasting avec variables exogènes

### Exercice 3 - Détection d'Anomalies
- Modèle entraîné sur patterns normaux
- Anomalies = erreurs de prédiction élevées
- Seuil basé sur distribution des erreurs
- Trade-off précision/rappel selon seuil

**Points clés généraux :**
- Toujours visualiser les données d'abord
- Time series split (jamais CV classique)
- Normalisation sur train uniquement
- Métriques appropriées (MAE, RMSE, MAPE)
- Choisir modèle selon nature des données